In [4]:
import cv2
import numpy as np

from keras.applications import imagenet_utils

import os

DataPath = '/Users/uttam/Downloads/Dataset/'
data_shape = 360*480

class Dataset:
    def __init__(self, classes=12, train_file='train.txt', test_file='test.txt'):
        self.train_file = train_file
        self.test_file = test_file
        self.data_shape = 360*480
        self.classes = classes

    def normalized(self, rgb):
        #return rgb/255.0
        norm=np.zeros((rgb.shape[0], rgb.shape[1], 3),np.float32)

        b=rgb[:,:,0]
        g=rgb[:,:,1]
        r=rgb[:,:,2]

        norm[:,:,0]=cv2.equalizeHist(b)
        norm[:,:,1]=cv2.equalizeHist(g)
        norm[:,:,2]=cv2.equalizeHist(r)

        return norm

    def one_hot_it(self, labels):
        x = np.zeros([360,480,12])
        for i in range(360):
            for j in range(480):
                x[i,j,labels[i][j]] = 1
        return x

    def load_data(self,):
        data = []
        label = []
       
        
        data.append(self.normalized(cv2.imread('/Users/uttam/Downloads/Data 2/img1.jpg')))
        label.append(self.one_hot_it(cv2.imread('/Users/uttam/Downloads/i2.png')[:,:,0]))
        return np.array(data), np.array(label)
        
        
       

        


    def preprocess_inputs(self, X):
    ### @ https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py
        """Preprocesses a tensor encoding a batch of images.
        # Arguments
            x: input Numpy tensor, 4D.
            data_format: data format of the image tensor.
            mode: One of "caffe", "tf".
                - caffe: will convert the images from RGB to BGR,
                    then will zero-center each color channel with
                    respect to the ImageNet dataset,
                    without scaling.
                - tf: will scale pixels between -1 and 1,
                    sample-wise.
        # Returns
            Preprocessed tensor.
        """
        return imagenet_utils.preprocess_input(X)

    def reshape_labels(self, y):
        return np.reshape(y, (len(y), self.data_shape, self.classes))

In [42]:
from keras.layers import Input
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.utils import np_utils

def SegNet(input_shape=(360, 480, 3), classes=12):
    ### @ https://github.com/alexgkendall/SegNet-Tutorial/blob/master/Example_Models/bayesian_segnet_camvid.prototxt
    img_input = Input(shape=input_shape)
    x = img_input
    # Encoder
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Decoder
    x = Conv2D(512, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(classes, (1, 1), padding="valid")(x)
    x = Reshape((input_shape[0] * input_shape[1], classes))(x)
    x = Activation("softmax")(x)
    model = Model(img_input, x)
    return model

In [59]:
import numpy as np
import keras
from PIL import Image


def calarea(arr):
    count=0
    for i in range(360):
        for j in range(480):
            if(np.array_equal(arr[i][j],[237,177,31])):
                count=count+1
    print(1-count/(360*480))             
          
height = 360
width = 480
classes = 3
epochs = 100
batch_size = 1
log_filepath='./logs_100/'

data_shape = 360*480
count=0

def writeImage(image, filename):
    """ label data to colored image """
    Water = [31,177,237]
    Land =  [31,177,237]
    Undefined=[189,113,0] 
    r = image.copy()
    g = image.copy()
    b = image.copy()
    label_colours = np.array([Water,Land,Undefined])
    for l in range(0,3):
        r[image==l] = label_colours[l,0]
        g[image==l] = label_colours[l,1]
        b[image==l] = label_colours[l,2]
    rgb = np.zeros((image.shape[0], image.shape[1], 3))
    rgb[:,:,0] = r/1.0
    rgb[:,:,1] = g/1.0
    rgb[:,:,2] = b/1.0
    im = Image.fromarray(np.uint8(rgb))
    im.save(filename)
    return

def predictveg(test):
    model = keras.models.load_model('/Users/uttam/Downloads/segvegetationf2.h5')
    probs = model.predict(test, batch_size=1)

    prob = probs[0].reshape((height, width, classes)).argmax(axis=2)
    return prob
def predictland(test):
    model = keras .models.load_model('/Users/uttam/Downloads/seg1.h5')
    probs = model.predict(test, batch_size=1)

    prob = probs[0].reshape((height, width, classes)).argmax(axis=2)
    return prob
def predictbarren(test):
    model = keras.models.load_model('/Users/uttam/Downloads/segbarren.h5')
    probs = model.predict(test, batch_size=1)

    prob = probs[0].reshape((height, width, classes)).argmax(axis=2)
    return prob
def predictroad(test):
    model = keras.models.load_model('/Users/uttam/Downloads/seg2.h5')
    probs = model.predict(test, batch_size=1)

    prob = probs[0].reshape((height, width, classes)).argmax(axis=2)
    return prob
def predictweeds(test):
    model = keras.models.load_model('/Users/uttam/Downloads/segweed.h5')
    probs = model.predict(test, batch_size=1)

    prob = probs[0].reshape((height, width, classes)).argmax(axis=2)
    return prob

def main():
    print("loading data...")
    ds = Dataset(test_file='val.txt', classes=classes)
    test_X, test_y = ds.load_data() # need to implement, y shape is (None, 360, 480, classes)
    test_X = ds.preprocess_inputs(test_X)
    #test_Y = ds.reshape_labels(test_y)

    #prob =  predictland(test_X)
    prob1 = predictbarren(test_X)
    prob2 = predictweeds(test_X)
    prob3 = predictroad(test_X)
    prob4 = predictveg(test_X)
  
   
    

    
   
    writeImage(prob1, 'val1.png')
    writeImage(prob2, 'val2.png')
    writeImage(prob3, 'val3.png')
    writeImage(prob4, 'val4.png')
    
    print("barren land area ")
    calarea(cv2.imread('/Users/uttam/val1.png'))
    print("weed area")
    calarea(cv2.imread('/Users/uttam/val2.png'))
    print("road area")
    calarea(cv2.imread('/Users/uttam/val3.png'))
    print("vege area")
    calarea(cv2.imread('/Users/uttam/val4.png'))
    
    

if __name__ == '__main__':
    main()

loading data...
barren land area 
0.5889872685185185
weed area
0.12368634259259259
road area
0.19054398148148144
vege area
0.13252314814814814
